# Regresion Logistica: Deteccion de SPAM

En este ejercico se muestran los fundamentos de la regresion logistica, planteando uno de los primeros problemas que fueron solucionados mediante el uso ded tecnicas de Machine Learning: La deteccion de SPAM


##  Enunciado del ejercicio.
Se propone la construccion de un sistema de aprendizaje automatico capaz de predecir si un correo determinado se corresponde con un correo SPAM o no, para ello se utilizara el siguente DatSet:

##### [2007_TE _Public_Spam_Corpus (https://plg.uwaterloo.ca/~gvcormac/treccorpus07/)]
The corpus trec07p contains 75,419 messages:

    25220 ham
    50199 spam

These messages constitute all the messages delivered to a particular
server between these dates:

    Sun, 8 Apr 2007 13:07:21 -0400
    Fri, 6 Jul 2007 07:04:53 -0400

In [2]:
# En esta clase se facilita el procesamiento de correos electronicos 

# que poseen codigo html.
from html.parser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.strict = False
        self.convert_charrefs = True
        self.fed = []

    def handle_data(self, d):
        self.fed.append(d)

    def get_data(self):
        return ''.join(self.fed)

In [3]:
# Esta funcion se encarga de eliminar los tags HTML
# que se encunetren en el texto de los correos electronicos
def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [4]:
# Ejemplo de eliminacion de los tads HTML de un texto
t = '<tr><td align="left"><ahref="../../issues/51/16.html#article">Phrack world News </a><td>'
strip_tags(t)

'Phrack world News '

Ademas de eliminar los posiblrs tags html que se encuentran en el correo electronico deben realizarse otras acciones para evitar que los mensajes contengan ruido inecesario. Entre ellas se encuentra la eliminacion de signos de puntuacion, eliminancion de los posibles campos de correo electronico que no sean relevantes o eliminacion de los afijos de una palabra manteniendo unicamente la raiz de la misma(stemming). La clase que se muestra a continuacion realiza estas transformaciones.

In [6]:
import email
import string
import nltk


class Parser:
    def __init__(self):
        self.stemmer = nltk.PorterStemmer()
        self.stopwords = set(nltk.corpus.stopwords.words('english'))

        self.punctuation = list(string.punctuation)

    def parse(self, email_path):
        """Parse an email."""
        with open(email_path, errors = 'ignore') as e:
            msg = email.message_from_file(e)
        return None if not msg else self.get_email_content(msg)

    def get_email_content(self, msg):
        """Extract the email content."""
        subject = self.tokenize(msg['Subject']) if msg ['Subject'] else []
        body = self.get_email_body(msg.get_payload(),
                                  msg.get_content_type())
        content_type = msg.get_content_type()
        # Return the content of the email
        return {"subject": subject,
               "body": body,
               "content_type": content_type}

    def get_email_body(self, payload, content_type):
        """Extract the body of the email."""
        body = []
        if type(payload) is str and content_type == 'text/plain':
            return self.tokenize(payload)
        elif type(payload) is str and content_type == 'text/html':
            return self.tokenize(strip_tags(payload))
        elif type(payload) is list:
            for p in payload:
                body += self.get_email_body(p.get_payload(), 
                                           p.get_content_type())
        return body

    def tokenize(self, text):
        """Transform a text string in tokens. Perform two main actons,
        clean the puntuaction symbols and do stemming of the text"""
        for c in self.punctuation:
            text = text.replace(c, "")
        text = text.replace("\t", " ")
        text = text.replace("\n", " ")
        tokens = list(filter(None, text.split(" ")))
        # Stremming of the tokens
        return [self.stemmer.stem(w) for w in tokens if w not in self.stopwords]

Lectura de un correo en formato .raw

In [ ]:
inmail = open("datasets/datasets/trec07p/data/emails.csv").read()
print(inmail)

##### Parsing del correo electronio

In [ ]:
p = Parser()
p.parse("datasets/datasets/trec07p/data/inmail.1")

##### Lectura del indice
Estas funciones complementarias se encargan de cargar en memoria la ruta de cada correo electronico y su etiqueta correspondinete.
{Spam,ham}

In [ ]:
index = open("datasets/datasets/trec07p/full/index").readlines()
index

In [ ]:
import os

DATASET_PATH = "datasets/datasets/trec07p"

def parse_index(path_to_index, n_elements):
    ret_indexes = []
    index = open(path_to_index).readlines()
    for i in range(n_elements):
        mail = index[i].split(" ../")
        label = mail[0]
        path = mail[1].strip() 
        ret_indexes.append({"label": label, "email_path": os.path.join(DATASET_PATH, path)})

    return ret_indexes


In [ ]:
def parse_email(index):
    p = Parser()
    pmail = p.parse(index["email_path"])
    return pmail, index["label"]

In [ ]:
indexes = parse_index("datasets/datasets/trec07p/full/index", 10)
indexes

##### Prepocesamiento del DataSet.

Con las funciones presentadas anteriormente se permite la lectura de los codigos electronicos de manera programatica  y el procesamineto de los mismos para eliminar aquellos componentes componentes que no resultan de utilidad para la deteccion de correos de SPAM. Sin embargo, cada uno de los correos sigue estando representado por un diccionario de python con una serie de palabras

In [ ]:
# Cargar el indice y las etiquetas en memoria
index = parse_index("datasets/datasets/trec07p/full/index", 1)

In [ ]:
# Leemos primer correo

import os

open(index[0]["email_path"]).read()

In [ ]:
# Parsear el primer correo
mail, label = parse_email(index[0])
print("El correo es: \n",label)
print(mail)

El algoritmo de Regresion LogÍstica no es capaz de ingerir texto como parte del DataSet. Por lo tanto deben de aplicarse una serie de funciones adicionales que transformen el texto de los correos electrónicos parseados en una representación númerica

### Aplicacion de countVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

# Preparacion del email en una cadena de texto.
prep_email = [" ".join(mail['subject']) + " ".join(mail['body'])]

vectorizer = CountVectorizer()
X = vectorizer.fit(prep_email)

print("\n\ne-mail:", prep_email, "\n")
print("Caracteristicas de entrada:", vectorizer.get_feature_names_out())

In [ ]:
X = vectorizer.transform(prep_email)
print("\nValues:\n", X.toarray())

#### Aplicacion de OneHotEncoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

prep_email = [[w] for w in mail['subject'] +mail['body']]
enc = OneHotEncoder(handle_unknown = 'ignore')
X = enc.fit_transform(prep_email)

print("Features:\n", enc.get_feature_names_out(), "\n")
print("Values:", X.toarray())

#### Funciones auxiliares para el procesamiento del DataSet

In [ ]:
def create_prep_dataset(index_path, n_elements):
    X = []
    y = []
    indexes = parse_index(index_path, n_elements)
    for i in range(n_elements):
        print("\rParsing email: {0}".format(i+1), end = '')
        mail, label = parse_email(indexes[i])
        X.append(" ".join(['subject']) + " ".join(mail['body']))
        y.append(label)
    return X,y

In [ ]:
# Leer unicamente un subconjunto de 1000 correos electronicos.
X_train, y_train = create_prep_dataset("datasets/datasets/trec07p/full/index", 1000)
X_train

##### Aplicar vectorizacion a los datos

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [ ]:
print(X_train.toarray())
print("\nFeatures", len(vectorizer.get_feature_names_out()))

In [ ]:
import pandas as pd

pd.DataFrame(X_train.toarray(), columns=[vectorizer.get_feature_names_out()])

In [ ]:
y_train

#### Entrenamiento del algoritmo de Regresion Logistica con en DataSet preprocesado

In [ ]:
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train, y_train)

# 4.- Prediccion

In [ ]:
# Lectura de un DataSet de correos nuevos.

# Leer 1500 correos de nuestro DataSet y quedarnos unicamente con los 500 ultimos correos electronicos, los cuales no se han utilizado
# Para entrenar el  algoritmo
X,y = create_prep_dataset("datasets/datasets/trec07p/full/index", 150)
X_test = X[100:]
y_test = y[100:]

##### Preprpcesamiento de los correos electronicos con el vectorizado creado anteriormenete 

In [ ]:
X_test =  vectorizer.transform(X_test)

In [ ]:
y_pred = clf.predict(X_test)
y_pred

In [ ]:
print("Prediccion\n", y_pred)
print("\nEtiquetas Reales", y_test)

#### Evaluacion de Resultados

In [ ]:
from sklearn.metrics import accuracy_score
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))

# 5.- Aumentando el DataSet

In [ ]:
# Leer 20000 correos electronicos
X, y = create_prep_dataset("datasets/datasets/trec07p/full/index", 20000)

In [ ]:
# Utilizamos 15,000 para entrenar el algoritmo y 5,000 para realizar proebas
X_train, y_train = X[:15000], y[:15000]
X_test, y_test = X[15000:], y[15000:]

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [ ]:
clf = LogisticRegression()
clf.fit(X_train, y_train)

In [ ]:
X_test = vectorizer.transform(X_test)
y_pred = clf.predict(X_test)

In [ ]:
print("Accuracy: {:.3f}".format(accuracy_score(y_test, y_pred)))